In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from utils.adaptacion import *
from keras.models import load_model


import os
import time
import pandas as pd
import numpy as np
from sklearn.externals import joblib

#variables para guardar los resultados
tipo = pruebas[4]
dataset_name = datasets[1]
dims = dimensions[dataset_name]

Using Theano backend.
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# Dataset de Twitter

In [3]:
print tipo
print dataset_name
print data_path
print dims

sda
twitter
data
1000


In [4]:
# cargando dataset de Twitter
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [5]:
# se obtienen todos los valores X disponibles para realizar adaptacion
X = dataset_object.get_all_X()
X = np.asarray(X)

print "Todos los datos disponibles obtenidos"

Todos los datos disponibles obtenidos


In [6]:
sda_paths = os.path.join(models_path, tipo, "%s_paths.pkl" % dataset_name)

if os.path.exists(sda_paths):
    print "Cargando rutas de modelos adaptados."
    saved_paths = joblib.load(sda_paths)
else:
    print "Creando modelos de adaptacion..."

    noises = [0.5, 0.8]
    layers = [[int(dims/2)], [int(dims/2), int(dims/4)], [int(dims/2), int(dims/4), int(dims/5)]]
    epochs = [50, 100, 250]

    parametros = {
        'noise': noises,
        'layers': layers,
        'epochs': epochs,
    }
    
    saved_paths = sda_pseudo_grid_search(X, parametros, models_path, tipo, dataset_name)

    print "Modelos creados.\n"
    
    joblib.dump(saved_paths, sda_paths)
    print "Guardando rutas en %s" % sda_paths

Creando modelos de adaptacion...
pr: 0.500 - epochs: 50 - layers: [500]
	Entrenando autoencoder...
	Guardando autoencoder en models/sda/twitter_ae_0.h5
	Guardando encoder en models/sda/twitter_e_0.h5
pr: 0.500 - epochs: 100 - layers: [500]
	Entrenando autoencoder...
	Guardando autoencoder en models/sda/twitter_ae_1.h5
	Guardando encoder en models/sda/twitter_e_1.h5
pr: 0.500 - epochs: 250 - layers: [500]
	Entrenando autoencoder...
	Guardando autoencoder en models/sda/twitter_ae_2.h5
	Guardando encoder en models/sda/twitter_e_2.h5
pr: 0.500 - epochs: 50 - layers: [500, 250]
	Entrenando autoencoder...
	Guardando autoencoder en models/sda/twitter_ae_3.h5
	Guardando encoder en models/sda/twitter_e_3.h5
pr: 0.500 - epochs: 100 - layers: [500, 250]
	Entrenando autoencoder...
	Guardando autoencoder en models/sda/twitter_ae_4.h5
	Guardando encoder en models/sda/twitter_e_4.h5
pr: 0.500 - epochs: 250 - layers: [500, 250]
	Entrenando autoencoder...
	Guardando autoencoder en models/sda/twitter_ae

In [7]:
saved_paths

[{'autoencoder': 'models/sda/twitter_ae_0.h5',
  'encoder': 'models/sda/twitter_e_0.h5'},
 {'autoencoder': 'models/sda/twitter_ae_1.h5',
  'encoder': 'models/sda/twitter_e_1.h5'},
 {'autoencoder': 'models/sda/twitter_ae_2.h5',
  'encoder': 'models/sda/twitter_e_2.h5'},
 {'autoencoder': 'models/sda/twitter_ae_3.h5',
  'encoder': 'models/sda/twitter_e_3.h5'},
 {'autoencoder': 'models/sda/twitter_ae_4.h5',
  'encoder': 'models/sda/twitter_e_4.h5'},
 {'autoencoder': 'models/sda/twitter_ae_5.h5',
  'encoder': 'models/sda/twitter_e_5.h5'},
 {'autoencoder': 'models/sda/twitter_ae_6.h5',
  'encoder': 'models/sda/twitter_e_6.h5'},
 {'autoencoder': 'models/sda/twitter_ae_7.h5',
  'encoder': 'models/sda/twitter_e_7.h5'},
 {'autoencoder': 'models/sda/twitter_ae_8.h5',
  'encoder': 'models/sda/twitter_e_8.h5'},
 {'autoencoder': 'models/sda/twitter_ae_9.h5',
  'encoder': 'models/sda/twitter_e_9.h5'},
 {'autoencoder': 'models/sda/twitter_ae_10.h5',
  'encoder': 'models/sda/twitter_e_10.h5'},
 {'autoe

In [8]:
best_sda_paths = os.path.join(models_path, tipo, "best_%s_paths.pkl" % dataset_name )
if os.path.exists(best_sda_paths):
    #se cargan las rutas de los modelos guardados
    domain_sda_paths = joblib.load(best_sda_paths)   
else:
    domain_sda_paths = []
    #por cada dominio
    for domain in domains:
        #por cada modelo
        print "Obteniendo mejor clasificador para %s..." % domain
        
        i = 1
        best_score = 0
        best_encoder = None
        
        for temp_path in saved_paths:
            #autoencoder = load_model(temp_path['autoencoder'])
            encoder = load_model(temp_path['encoder'])

        
            X_tr = np.asarray(labeled[domain]['X_tr'].todense())
            y_tr = np.asarray(labeled[domain]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[domain]['X_ts'].todense())
            
            #se adapta el dominio segun el modelo
            X_deep = encoder.predict(X_tr)

            #se crea un clasificador y se obtiene su score
            new_clf = get_best_score(X_deep, y_tr, clasifier='SVC')
            new_score = new_clf.best_score_
            
            print "%d - score: %.4f" % (i, new_score)
            # se guarda si es el mejor para el modelo
            if new_score > best_score:
                best_score = new_score
                best_encoder = encoder
                
            i= i+1
            
        best_encoder_path = os.path.join(models_path, tipo, "%s_%s_encoder.pkl" % (dataset_name, domain))
        print "Guardando en %s" % best_encoder_path
        encoder.save(best_encoder_path)
        
        domain_sda_paths.append(best_encoder_path)
        
    print "Guardando rutas en %s" % best_sda_paths
    joblib.dump(domain_sda_paths, best_sda_paths)

Obteniendo mejor clasificador para rio2016...


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/keras/models.py:240: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


1 - score: 0.7947
2 - score: 0.7927
3 - score: 0.8100


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


4 - score: 0.7748
5 - score: 0.8033
6 - score: 0.7796
7 - score: 0.7313
8 - score: 0.7458
9 - score: 0.7674
10 - score: 0.7931
11 - score: 0.7864
12 - score: 0.8004
13 - score: 0.7630
14 - score: 0.7508
15 - score: 0.7218
16 - score: 0.7629
17 - score: 0.7458
18 - score: 0.7078
Guardando en models/sda/twitter_rio2016_encoder.pkl
Obteniendo mejor clasificador para thevoice...
1 - score: 0.7956
2 - score: 0.8010
3 - score: 0.8050
4 - score: 0.7540
5 - score: 0.7580
6 - score: 0.7643
7 - score: 0.7328
8 - score: 0.7138
9 - score: 0.7130
10 - score: 0.7889
11 - score: 0.7941
12 - score: 0.7815
13 - score: 0.7511
14 - score: 0.7492
15 - score: 0.7495
16 - score: 0.7542
17 - score: 0.7180
18 - score: 0.7268
Guardando en models/sda/twitter_thevoice_encoder.pkl
Guardando rutas en models/sda/best_twitter_paths.pkl


In [9]:
domain_sda_paths

['models/sda/twitter_rio2016_encoder.pkl',
 'models/sda/twitter_thevoice_encoder.pkl']

In [10]:
#diccionario para mantener los dominios adaptados
adapted = {}

i = 0
for domain in domains:
    best_sda_path = os.path.join(models_path, tipo, "%s_%s_encoder.pkl" % (dataset_name, domain))
    
    if os.path.exists(best_sda_path):
        print "Cargando mejor modelo para %s" % domain
        encoder = load_model(best_sda_path)
        
        print "Adaptando dominio"
        X_tr = np.asarray(labeled[domain]['X_tr'].todense())
        X_ts = np.asarray(labeled[domain]['X_ts'].todense())
        
        
        tr_reps = encoder.predict(X_tr)
        ts_reps = encoder.predict(X_ts)

        adapted[domain] = {
            'X_tr': tr_reps,
            'X_ts': ts_reps
        }
        
        i = i+1
    else:
        print "Generar mejor modelo para %s" % domain
        
print "%d/%d dominios adaptados" % (i, len(domains))

Cargando mejor modelo para rio2016
Adaptando dominio
Cargando mejor modelo para thevoice
Adaptando dominio
2/2 dominios adaptados


In [11]:
df = pd.DataFrame(columns=dataframe_columns)

i = 0
# por cada par posible para adaptar
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de 2" % (i+1)
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'][:, :dims].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'][:, :dims].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            #model_name = "%s_%s_%s.pkl" % (tipo,src, tgt)
            model_name = "indomain_%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, model_name)
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            #transfer error
            #entrenado en dominio src y probado en dominio tgt adaptados
            X_tr_a = adapted[src]['X_tr']
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts_a = adapted[tgt]['X_ts']
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            print "Entrenando clasificador adaptado."
            svc_a = get_best_score(X_tr_a, y_tr)
            
            t_error = 1-svc_a.score(X_ts_a, y_ts)
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['SDA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            i+=1

print "Pruebas completadas."

Tarea 1 de 2
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 2 de 2
Cargando modelo existente.
Entrenando clasificador adaptado.
Pruebas completadas.


In [12]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,SDA,r->t,rio2016,thevoice,15.351812,31.878788,16.526976
1,SDA,t->r,thevoice,rio2016,19.642857,38.430913,18.788056


In [13]:
new_scores_path = os.path.join(scores_path, tipo, dataset_name+".csv")

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."


Guardando en scores/sda/twitter.csv
Resultados guardados.
